# Highway with SB3's DQN

##  Warming up
We start with a few useful installs and imports:

In [3]:
# Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/DLR-RM/stable-baselines3

# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import DQN

# Visualization utils
%load_ext tensorboard
import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!git clone https://github.com/eleurent/highway-env.git 2> /dev/null
sys.path.insert(0, '/content/highway-env/scripts/')
from utils import record_videos, show_videos

Processing /home/azureuser/.cache/pip/wheels/17/79/65/7afedc162d858b02708a3b8f7a6dd5b1000dcd5b0f894f7cc1/gym-0.26.2-py3-none-any.whl
ERROR: stable-baselines3 1.7.0a0 has requirement gym==0.21, but you'll have gym 0.26.2 which is incompatible.
  Attempting uninstall: gym
    Found existing installation: gym 0.21.0
    Uninstalling gym-0.21.0:
      Successfully uninstalled gym-0.21.0
  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-5qyrjtow
  Running command git clone -q https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-5qyrjtow
Processing /home/azureuser/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73/gym-0.21.0-py3-none-any.whl
  Created wheel for stable-baselines3: filename=stable_baselines3-1.7.0a0-py3-none-any.whl size=166975 sha256=c7df49c29cad53f0dd5e514b75771552518154b1620b342b64e3ad0b2be15f9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-na79lwn9/wheels/90/4e/06/447116d15ce3af2647603a9b5c7c14b5148fd800bba

## Training
Run tensorboard locally to visualize training.

In [10]:
%tensorboardx --logdir "highway_dqn"

UsageError: Line magic function `%tensorboardx` not found.


In [5]:
model = DQN('MlpPolicy', "highway-fast-v0",
                policy_kwargs=dict(net_arch=[256, 256]),
                learning_rate=5e-4,
                buffer_size=15000,
                learning_starts=200,
                batch_size=32,
                gamma=0.8,
                train_freq=1,
                gradient_steps=1,
                target_update_interval=50,
                exploration_fraction=0.7,
                verbose=1,
                tensorboard_log="highway_dqn/")
model.learn(int(2e4))


Using cpu device
Creating environment from the given name 'highway-fast-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to highway_dqn/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 13.5     |
|    ep_rew_mean      | 10.3     |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 13       |
|    time_elapsed     | 4        |
|    total_timesteps  | 54       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 11.6     |
|    ep_rew_mean      | 8.54     |
|    exploration_rate | 0.994    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 15       |
|    time_elapsed     | 5        |
|    total_timesteps  | 93       |
----------------------------------
----------------------------------
| rollout/   

## Testing

Visualize a few episodes

In [9]:
model.save("asdf")

In [8]:
env = gym.make("highway-fast-v0")
env = record_videos(env)
for episode in trange(3, desc="Test episodes"):
    obs, done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(int(action))
        # env.render()
env.close()
show_videos()


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]